In [1]:
#Basic Imports
from __future__ import absolute_import,division,print_function
import numpy as np
np.random.seed(1337)
from matplotlib import pyplot as plt
%matplotlib inline
from os.path import isfile

<h1>Transfer Learning Experiment with Indic Scripts</h1>
<h2>Abdul Kawsar Tushar <br> Akm Ashiquzzaman<br>Afia Afrin </h2>

Fall 2017

<p> About transfer Learning: <br> 

</p>
<p> Indic Scripts and How transfer learning will help: <br>
    
</p>

In [2]:
#Now Let's define the model
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,MaxPool2D,Activation,Flatten
from keras.optimizers import Adam,Adadelta
from keras.losses import categorical_crossentropy
from keras import utils
from keras.callbacks import History,ModelCheckpoint

Using TensorFlow backend.


Now let's think of the model a little bit. As suggested by AKT, the model will be followed by the pattren of imagnet, or the arabnet v2. The main idea was to see the transfer learning in classification. So the model can be divided into 2 part. 

1. Feature extraction Part.
2. Classification Part.

We need the weight of the former part to be freezed in backprop.  


In [3]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Conv2D(64,3,
           padding='valid',use_bias=False,
           input_shape=(1,32,32)),
    Activation('elu'),
    Conv2D(64,3,use_bias=False),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(64,3,use_bias=False),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(32,3,use_bias=False),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(512,use_bias=False),
    Activation('elu'),
    Dropout(0.5),
    Dense(256,use_bias=False),
    Activation('elu'),
    Dropout(0.5),
    Dense(128,use_bias=False),
    Activation('elu'),
    Dropout(0.5),
    Dense(10,use_bias=False),
    Activation('softmax')
]

#model building
model = Sequential(feature_layers + classification_layers)

In [4]:
#Now let's process arabic datas.
Arabic_datX = np.load('Arabic.npz')['arr_0']
Arabic_datY = np.load('Arabic.npz')['arr_1']

Arabic_datX = (Arabic_datX/255).astype('float32')
Arabic_datX = Arabic_datX.reshape((3000,1,32,32))
Arabic_datY = utils.np_utils.to_categorical(Arabic_datY,10)

Arabic_datX.shape,Arabic_datY.shape

((3000, 1, 32, 32), (3000, 10))

In [5]:
model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 30, 30)        576       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 30, 30)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 28, 28)        36864     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 28, 28)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 28, 28)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 26, 26)        36864     
_________________________________________________________________
activation_3 (Activation)    (None, 64, 26, 26)        0         
__________

In [9]:
#MLP weight flepath saving, we're usung the callback loss to save the best weight model.  
weight_path="Arabic-transfer-main-weight.h5"

if(isfile(weight_path)):
    model.load_weights(weight_path)

checkpoint = ModelCheckpoint(weight_path, monitor='acc', verbose=1, save_best_only=True, mode='max')

#Some other parameter will be added later for finetune
callbacks_list = [checkpoint]

hist = model.fit(Arabic_datX, Arabic_datY, batch_size=100, epochs=300,
          verbose=1,validation_split=.3,callbacks=callbacks_list)


Train on 2100 samples, validate on 900 samples
Epoch 1/300
2100/2100 [==============================] - 10s - loss: 1.7313 - acc: 0.4048 - val_loss: 0.9358 - val_acc: 0.7333
Epoch 2/300
2100/2100 [==============================] - 1s - loss: 0.8764 - acc: 0.6948 - val_loss: 0.5389 - val_acc: 0.8322
Epoch 3/300
2100/2100 [==============================] - 1s - loss: 0.6329 - acc: 0.7814 - val_loss: 0.5212 - val_acc: 0.8322
Epoch 4/300
2100/2100 [==============================] - 1s - loss: 0.4886 - acc: 0.8352 - val_loss: 0.4366 - val_acc: 0.8622
Epoch 5/300
2100/2100 [==============================] - 1s - loss: 0.4360 - acc: 0.8614 - val_loss: 0.4186 - val_acc: 0.8589
Epoch 6/300
2100/2100 [==============================] - 1s - loss: 0.4005 - acc: 0.8652 - val_loss: 0.4462 - val_acc: 0.8544
Epoch 7/300
2100/2100 [==============================] - 1s - loss: 0.3626 - acc: 0.8757 - val_loss: 0.3676 - val_acc: 0.8878
Epoch 8/300
2100/2100 [==============================] - 1s - loss: 0.

2100/2100 [==============================] - 1s - loss: 0.0332 - acc: 0.9905 - val_loss: 0.3912 - val_acc: 0.9311
Epoch 60/300
2100/2100 [==============================] - 1s - loss: 0.0321 - acc: 0.9933 - val_loss: 0.3430 - val_acc: 0.9322
Epoch 61/300
2100/2100 [==============================] - 1s - loss: 0.0479 - acc: 0.9857 - val_loss: 0.3569 - val_acc: 0.9244
Epoch 62/300
2100/2100 [==============================] - 1s - loss: 0.0313 - acc: 0.9895 - val_loss: 0.3752 - val_acc: 0.9333
Epoch 63/300
2100/2100 [==============================] - 1s - loss: 0.0294 - acc: 0.9895 - val_loss: 0.3657 - val_acc: 0.9311
Epoch 64/300
2100/2100 [==============================] - 1s - loss: 0.0340 - acc: 0.9876 - val_loss: 0.4258 - val_acc: 0.9178
Epoch 65/300
2100/2100 [==============================] - 1s - loss: 0.0411 - acc: 0.9862 - val_loss: 0.4080 - val_acc: 0.9233
Epoch 66/300
2100/2100 [==============================] - 1s - loss: 0.0268 - acc: 0.9914 - val_loss: 0.3799 - val_acc: 0.93

2100/2100 [==============================] - 1s - loss: 0.0202 - acc: 0.9962 - val_loss: 0.3808 - val_acc: 0.9444
Epoch 92/300
2100/2100 [==============================] - 1s - loss: 0.0215 - acc: 0.9924 - val_loss: 0.3939 - val_acc: 0.9411
Epoch 93/300
2100/2100 [==============================] - 1s - loss: 0.0291 - acc: 0.9914 - val_loss: 0.3672 - val_acc: 0.9478
Epoch 94/300
2100/2100 [==============================] - 1s - loss: 0.0232 - acc: 0.9943 - val_loss: 0.3433 - val_acc: 0.9478
Epoch 95/300
2100/2100 [==============================] - 1s - loss: 0.0149 - acc: 0.9952 - val_loss: 0.3803 - val_acc: 0.9356
Epoch 96/300
2100/2100 [==============================] - 1s - loss: 0.0164 - acc: 0.9957 - val_loss: 0.3612 - val_acc: 0.9400
Epoch 97/300
2100/2100 [==============================] - 1s - loss: 0.0158 - acc: 0.9952 - val_loss: 0.3726 - val_acc: 0.9467
Epoch 98/300
2100/2100 [==============================] - 1s - loss: 0.0244 - acc: 0.9924 - val_loss: 0.4140 - val_acc: 0.93

2100/2100 [==============================] - 1s - loss: 0.0104 - acc: 0.9962 - val_loss: 0.4257 - val_acc: 0.9422
Epoch 125/300
2100/2100 [==============================] - 1s - loss: 0.0160 - acc: 0.9957 - val_loss: 0.4353 - val_acc: 0.9444
Epoch 126/300
2100/2100 [==============================] - 1s - loss: 0.0139 - acc: 0.9967 - val_loss: 0.4077 - val_acc: 0.9422
Epoch 127/300
2100/2100 [==============================] - 1s - loss: 0.0191 - acc: 0.9948 - val_loss: 0.3887 - val_acc: 0.9433
Epoch 128/300
2100/2100 [==============================] - 1s - loss: 0.0156 - acc: 0.9962 - val_loss: 0.3880 - val_acc: 0.9400
Epoch 129/300
2100/2100 [==============================] - 1s - loss: 0.0122 - acc: 0.9962 - val_loss: 0.3827 - val_acc: 0.9356
Epoch 130/300
2100/2100 [==============================] - 1s - loss: 0.0072 - acc: 0.9971 - val_loss: 0.3820 - val_acc: 0.9389
Epoch 131/300
2100/2100 [==============================] - 1s - loss: 0.0214 - acc: 0.9933 - val_loss: 0.3903 - val_ac

2100/2100 [==============================] - 1s - loss: 0.0065 - acc: 0.9981 - val_loss: 0.4001 - val_acc: 0.9489
Epoch 192/300
2100/2100 [==============================] - 1s - loss: 0.0081 - acc: 0.9981 - val_loss: 0.4366 - val_acc: 0.9433
Epoch 193/300
2100/2100 [==============================] - 1s - loss: 0.0205 - acc: 0.9938 - val_loss: 0.4377 - val_acc: 0.9400
Epoch 194/300
2100/2100 [==============================] - 1s - loss: 0.0119 - acc: 0.9976 - val_loss: 0.4314 - val_acc: 0.9411
Epoch 195/300
2100/2100 [==============================] - 1s - loss: 0.0112 - acc: 0.9976 - val_loss: 0.4108 - val_acc: 0.9456
Epoch 196/300
2100/2100 [==============================] - 1s - loss: 0.0123 - acc: 0.9957 - val_loss: 0.4578 - val_acc: 0.9400
Epoch 197/300
2100/2100 [==============================] - 1s - loss: 0.0165 - acc: 0.9952 - val_loss: 0.4283 - val_acc: 0.9433
Epoch 198/300
2100/2100 [==============================] - 1s - loss: 0.0122 - acc: 0.9962 - val_loss: 0.4244 - val_ac

2100/2100 [==============================] - 1s - loss: 0.0077 - acc: 0.9981 - val_loss: 0.5229 - val_acc: 0.9367
Epoch 259/300
2100/2100 [==============================] - 1s - loss: 0.0131 - acc: 0.9971 - val_loss: 0.5328 - val_acc: 0.9400
Epoch 260/300
2100/2100 [==============================] - 1s - loss: 0.0093 - acc: 0.9971 - val_loss: 0.5439 - val_acc: 0.9400
Epoch 261/300
2100/2100 [==============================] - 1s - loss: 0.0044 - acc: 0.9986 - val_loss: 0.5379 - val_acc: 0.9433
Epoch 262/300
2100/2100 [==============================] - 1s - loss: 0.0069 - acc: 0.9981 - val_loss: 0.5129 - val_acc: 0.9478
Epoch 263/300
2100/2100 [==============================] - 1s - loss: 8.2341e-04 - acc: 1.0000 - val_loss: 0.5031 - val_acc: 0.9444
Epoch 264/300
2100/2100 [==============================] - 1s - loss: 0.0086 - acc: 0.9981 - val_loss: 0.4899 - val_acc: 0.9433
Epoch 265/300
2100/2100 [==============================] - 1s - loss: 0.0084 - acc: 0.9976 - val_loss: 0.5216 - va

In [5]:
#Now let's process Bangla datas.
Bangla_datX = np.load('Bangali.npz')['arr_0']
Bangla_datY = np.load('Bangali.npz')['arr_1']

Bangla_datX = (Bangla_datX/255).astype('float32')
Bangla_datX = Bangla_datX.reshape((Bangla_datX.shape[0],1,32,32))
Bangla_datY = utils.np_utils.to_categorical(Bangla_datY,10)

Bangla_datX.shape,Bangla_datY.shape

((6000, 1, 32, 32), (6000, 10))

In [10]:
model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
#MLP weight flepath saving, we're usung the callback loss to save the best weight model.  
weight_path="Bangla-transfer-main-weight.h5"

if(isfile(weight_path)):
    model.load_weights(weight_path)

checkpoint = ModelCheckpoint(weight_path, monitor='acc', verbose=1, save_best_only=True, mode='max')

#Some other parameter will be added later for finetune
callbacks_list = [checkpoint]

hist = model.fit(Bangla_datX, Bangla_datY, batch_size=100, epochs=300,
          verbose=1,validation_split=.3,callbacks=callbacks_list)

Train on 4200 samples, validate on 1800 samples
Epoch 1/300
4200/4200 [==============================] - 2s - loss: 4.0695 - acc: 0.3431 - val_loss: 0.8014 - val_acc: 0.7472
Epoch 2/300
4200/4200 [==============================] - 2s - loss: 1.1843 - acc: 0.6710 - val_loss: 0.4744 - val_acc: 0.8572
Epoch 3/300
4200/4200 [==============================] - 2s - loss: 0.8065 - acc: 0.7688 - val_loss: 0.3925 - val_acc: 0.8800
Epoch 4/300
4200/4200 [==============================] - 2s - loss: 0.6370 - acc: 0.8200 - val_loss: 0.3324 - val_acc: 0.9039
Epoch 5/300
4200/4200 [==============================] - 2s - loss: 0.4579 - acc: 0.8638 - val_loss: 0.3042 - val_acc: 0.9217
Epoch 6/300
4200/4200 [==============================] - 2s - loss: 0.4248 - acc: 0.8729 - val_loss: 0.2670 - val_acc: 0.9322
Epoch 7/300
4200/4200 [==============================] - 2s - loss: 0.3328 - acc: 0.8950 - val_loss: 0.2519 - val_acc: 0.9333
Epoch 8/300
4200/4200 [==============================] - 2s - loss: 0.

4200/4200 [==============================] - 2s - loss: 0.0195 - acc: 0.9955 - val_loss: 0.3547 - val_acc: 0.9589
Epoch 61/300
4200/4200 [==============================] - 2s - loss: 0.0177 - acc: 0.9955 - val_loss: 0.3373 - val_acc: 0.9578
Epoch 62/300
4200/4200 [==============================] - 2s - loss: 0.0151 - acc: 0.9955 - val_loss: 0.3519 - val_acc: 0.9578
Epoch 63/300
4200/4200 [==============================] - 2s - loss: 0.0251 - acc: 0.9945 - val_loss: 0.3881 - val_acc: 0.9556
Epoch 64/300
4200/4200 [==============================] - 2s - loss: 0.0192 - acc: 0.9943 - val_loss: 0.3577 - val_acc: 0.9589
Epoch 65/300
4200/4200 [==============================] - 2s - loss: 0.0151 - acc: 0.9950 - val_loss: 0.3654 - val_acc: 0.9550
Epoch 66/300
4200/4200 [==============================] - 2s - loss: 0.0176 - acc: 0.9943 - val_loss: 0.3876 - val_acc: 0.9533
Epoch 67/300
4200/4200 [==============================] - 2s - loss: 0.0150 - acc: 0.9952 - val_loss: 0.3903 - val_acc: 0.95

4200/4200 [==============================] - 2s - loss: 0.0121 - acc: 0.9971 - val_loss: 0.4751 - val_acc: 0.9511
Epoch 127/300
4200/4200 [==============================] - 2s - loss: 0.0126 - acc: 0.9974 - val_loss: 0.4713 - val_acc: 0.9528
Epoch 128/300
4200/4200 [==============================] - 2s - loss: 0.0071 - acc: 0.9983 - val_loss: 0.4482 - val_acc: 0.9556
Epoch 129/300
4200/4200 [==============================] - 2s - loss: 0.0048 - acc: 0.9988 - val_loss: 0.4256 - val_acc: 0.9572
Epoch 130/300
4200/4200 [==============================] - 2s - loss: 0.0130 - acc: 0.9969 - val_loss: 0.4531 - val_acc: 0.9522
Epoch 131/300
4200/4200 [==============================] - 2s - loss: 0.0080 - acc: 0.9979 - val_loss: 0.4218 - val_acc: 0.9572
Epoch 132/300
4200/4200 [==============================] - 2s - loss: 0.0034 - acc: 0.9983 - val_loss: 0.4089 - val_acc: 0.9600
Epoch 133/300
4200/4200 [==============================] - 2s - loss: 0.0040 - acc: 0.9990 - val_loss: 0.4391 - val_ac

4200/4200 [==============================] - 2s - loss: 6.2567e-04 - acc: 0.9998 - val_loss: 0.4878 - val_acc: 0.9561
Epoch 194/300
4200/4200 [==============================] - 2s - loss: 0.0019 - acc: 0.9990 - val_loss: 0.5091 - val_acc: 0.9550
Epoch 195/300
4200/4200 [==============================] - 2s - loss: 9.3292e-04 - acc: 0.9995 - val_loss: 0.4788 - val_acc: 0.9572
Epoch 196/300
4200/4200 [==============================] - 2s - loss: 2.5545e-04 - acc: 0.9998 - val_loss: 0.5017 - val_acc: 0.9556
Epoch 197/300
4200/4200 [==============================] - 2s - loss: 0.0034 - acc: 0.9986 - val_loss: 0.4602 - val_acc: 0.9567
Epoch 198/300
4200/4200 [==============================] - 2s - loss: 0.0074 - acc: 0.9988 - val_loss: 0.5059 - val_acc: 0.9556
Epoch 199/300
4200/4200 [==============================] - 2s - loss: 0.0015 - acc: 0.9993 - val_loss: 0.4702 - val_acc: 0.9561
Epoch 200/300
4200/4200 [==============================] - 2s - loss: 0.0035 - acc: 0.9990 - val_loss: 0.4

4200/4200 [==============================] - 2s - loss: 0.0011 - acc: 0.9998 - val_loss: 0.5163 - val_acc: 0.9556
Epoch 227/300
4200/4200 [==============================] - 2s - loss: 0.0010 - acc: 0.9995 - val_loss: 0.5067 - val_acc: 0.9544
Epoch 228/300
4200/4200 [==============================] - 2s - loss: 0.0018 - acc: 0.9995 - val_loss: 0.4689 - val_acc: 0.9533
Epoch 229/300
4200/4200 [==============================] - 2s - loss: 0.0063 - acc: 0.9986 - val_loss: 0.4724 - val_acc: 0.9539
Epoch 230/300
4200/4200 [==============================] - 2s - loss: 0.0071 - acc: 0.9983 - val_loss: 0.4590 - val_acc: 0.9572
Epoch 231/300
4200/4200 [==============================] - 2s - loss: 0.0014 - acc: 0.9995 - val_loss: 0.4632 - val_acc: 0.9556
Epoch 232/300
4200/4200 [==============================] - 2s - loss: 8.1668e-04 - acc: 0.9998 - val_loss: 0.4310 - val_acc: 0.9567
Epoch 233/300
4200/4200 [==============================] - 2s - loss: 0.0018 - acc: 0.9990 - val_loss: 0.4258 - va

4200/4200 [==============================] - 2s - loss: 0.0053 - acc: 0.9993 - val_loss: 0.5279 - val_acc: 0.9539
Epoch 294/300
4200/4200 [==============================] - 2s - loss: 9.6620e-04 - acc: 0.9995 - val_loss: 0.4965 - val_acc: 0.9561
Epoch 295/300
4200/4200 [==============================] - 2s - loss: 9.9699e-04 - acc: 0.9998 - val_loss: 0.4944 - val_acc: 0.9544
Epoch 296/300
4200/4200 [==============================] - 2s - loss: 0.0042 - acc: 0.9988 - val_loss: 0.4885 - val_acc: 0.9561
Epoch 297/300
4200/4200 [==============================] - 2s - loss: 0.0014 - acc: 0.9998 - val_loss: 0.5137 - val_acc: 0.9572
Epoch 298/300
4200/4200 [==============================] - 2s - loss: 3.6138e-04 - acc: 0.9998 - val_loss: 0.5082 - val_acc: 0.9550
Epoch 299/300
4200/4200 [==============================] - 2s - loss: 8.4884e-04 - acc: 0.9993 - val_loss: 0.4744 - val_acc: 0.9600
Epoch 300/300
4200/4200 [==============================] - 2s - loss: 0.0031 - acc: 0.9995 - val_loss:

In [6]:
#Now let's process Hindi datas.
Hindi_datX = np.load('Hindi.npz')['arr_0']
Hindi_datY = np.load('Hindi.npz')['arr_1']

Hindi_datX = (Hindi_datX/255).astype('float32')
Hindi_datX = Hindi_datX.reshape((Hindi_datX.shape[0],1,32,32))
Hindi_datY = utils.np_utils.to_categorical(Hindi_datY,10)

Hindi_datX.shape,Hindi_datY.shape

((3000, 1, 32, 32), (3000, 10))

In [12]:
model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
#MLP weight flepath saving, we're usung the callback loss to save the best weight model.  
weight_path="Hindi-transfer-main-weight.h5"

if(isfile(weight_path)):
    model.load_weights(weight_path)

checkpoint = ModelCheckpoint(weight_path, monitor='acc', verbose=1, save_best_only=True, mode='max')

#Some other parameter will be added later for finetune
callbacks_list = [checkpoint]

hist = model.fit(Hindi_datX, Hindi_datY, batch_size=100, epochs=300,
          verbose=1,validation_split=.3,callbacks=callbacks_list)

Train on 2100 samples, validate on 900 samples
Epoch 1/300
2100/2100 [==============================] - 1s - loss: 4.9996 - acc: 0.3162 - val_loss: 1.2301 - val_acc: 0.5522
Epoch 2/300
2100/2100 [==============================] - 1s - loss: 1.9180 - acc: 0.4433 - val_loss: 0.8503 - val_acc: 0.7333
Epoch 3/300
2100/2100 [==============================] - 1s - loss: 1.4592 - acc: 0.5719 - val_loss: 0.5866 - val_acc: 0.8578
Epoch 4/300
2100/2100 [==============================] - 1s - loss: 1.2121 - acc: 0.6405 - val_loss: 0.5132 - val_acc: 0.8689
Epoch 5/300
2100/2100 [==============================] - 1s - loss: 1.0554 - acc: 0.6857 - val_loss: 0.3840 - val_acc: 0.9089
Epoch 6/300
2100/2100 [==============================] - 1s - loss: 0.8735 - acc: 0.7452 - val_loss: 0.3082 - val_acc: 0.9289
Epoch 7/300
2100/2100 [==============================] - 1s - loss: 0.7815 - acc: 0.7790 - val_loss: 0.3086 - val_acc: 0.9189
Epoch 8/300
2100/2100 [==============================] - 1s - loss: 0.6

2100/2100 [==============================] - 1s - loss: 0.1547 - acc: 0.9576 - val_loss: 0.1662 - val_acc: 0.9644
Epoch 29/300
2100/2100 [==============================] - 1s - loss: 0.1453 - acc: 0.9562 - val_loss: 0.1875 - val_acc: 0.9644
Epoch 30/300
2100/2100 [==============================] - 1s - loss: 0.1054 - acc: 0.9676 - val_loss: 0.1684 - val_acc: 0.9722
Epoch 31/300
2100/2100 [==============================] - 1s - loss: 0.1290 - acc: 0.9643 - val_loss: 0.1921 - val_acc: 0.9611
Epoch 32/300
2100/2100 [==============================] - 1s - loss: 0.1012 - acc: 0.9719 - val_loss: 0.2006 - val_acc: 0.9611
Epoch 33/300
2100/2100 [==============================] - 1s - loss: 0.1081 - acc: 0.9700 - val_loss: 0.1505 - val_acc: 0.9689
Epoch 34/300
2100/2100 [==============================] - 1s - loss: 0.1123 - acc: 0.9662 - val_loss: 0.1503 - val_acc: 0.9711
Epoch 35/300
2100/2100 [==============================] - 1s - loss: 0.0829 - acc: 0.9757 - val_loss: 0.1538 - val_acc: 0.96

2100/2100 [==============================] - 1s - loss: 0.0194 - acc: 0.9957 - val_loss: 0.2242 - val_acc: 0.9644
Epoch 93/300
2100/2100 [==============================] - 1s - loss: 0.0374 - acc: 0.9900 - val_loss: 0.2204 - val_acc: 0.9622
Epoch 94/300
2100/2100 [==============================] - 1s - loss: 0.0348 - acc: 0.9948 - val_loss: 0.2221 - val_acc: 0.9656
Epoch 95/300
2100/2100 [==============================] - 1s - loss: 0.0249 - acc: 0.9924 - val_loss: 0.2422 - val_acc: 0.9667
Epoch 96/300
2100/2100 [==============================] - 1s - loss: 0.0263 - acc: 0.9933 - val_loss: 0.2498 - val_acc: 0.9622
Epoch 97/300
2100/2100 [==============================] - 1s - loss: 0.0412 - acc: 0.9919 - val_loss: 0.1965 - val_acc: 0.9700
Epoch 98/300
2100/2100 [==============================] - 1s - loss: 0.0222 - acc: 0.9943 - val_loss: 0.2133 - val_acc: 0.9678
Epoch 99/300
2100/2100 [==============================] - 1s - loss: 0.0364 - acc: 0.9933 - val_loss: 0.2195 - val_acc: 0.95

2100/2100 [==============================] - 1s - loss: 0.0157 - acc: 0.9967 - val_loss: 0.2191 - val_acc: 0.9733
Epoch 159/300
2100/2100 [==============================] - 1s - loss: 0.0213 - acc: 0.9967 - val_loss: 0.2480 - val_acc: 0.9656
Epoch 160/300
2100/2100 [==============================] - 1s - loss: 0.0193 - acc: 0.9967 - val_loss: 0.2028 - val_acc: 0.9700
Epoch 161/300
2100/2100 [==============================] - 1s - loss: 0.0122 - acc: 0.9990 - val_loss: 0.2051 - val_acc: 0.9733
Epoch 162/300
2100/2100 [==============================] - 1s - loss: 0.0245 - acc: 0.9943 - val_loss: 0.1580 - val_acc: 0.9744
Epoch 163/300
2100/2100 [==============================] - 1s - loss: 0.0208 - acc: 0.9943 - val_loss: 0.2283 - val_acc: 0.9700
Epoch 164/300
2100/2100 [==============================] - 1s - loss: 0.0159 - acc: 0.9976 - val_loss: 0.2010 - val_acc: 0.9744
Epoch 165/300
2100/2100 [==============================] - 1s - loss: 0.0190 - acc: 0.9952 - val_loss: 0.1919 - val_ac

2100/2100 [==============================] - 1s - loss: 0.0240 - acc: 0.9952 - val_loss: 0.2496 - val_acc: 0.9667
Epoch 225/300
2100/2100 [==============================] - 1s - loss: 0.0050 - acc: 0.9986 - val_loss: 0.2348 - val_acc: 0.9700
Epoch 226/300
2100/2100 [==============================] - 1s - loss: 0.0102 - acc: 0.9981 - val_loss: 0.2080 - val_acc: 0.9689
Epoch 227/300
2100/2100 [==============================] - 1s - loss: 0.0057 - acc: 0.9990 - val_loss: 0.2101 - val_acc: 0.9678
Epoch 228/300
2100/2100 [==============================] - 1s - loss: 0.0038 - acc: 0.9986 - val_loss: 0.2585 - val_acc: 0.9667
Epoch 229/300
2100/2100 [==============================] - 1s - loss: 0.0070 - acc: 0.9995 - val_loss: 0.2968 - val_acc: 0.9622
Epoch 230/300
2100/2100 [==============================] - 1s - loss: 0.0066 - acc: 0.9986 - val_loss: 0.2745 - val_acc: 0.9678
Epoch 231/300
2100/2100 [==============================] - 1s - loss: 0.0089 - acc: 0.9976 - val_loss: 0.2657 - val_ac

2100/2100 [==============================] - 1s - loss: 0.0038 - acc: 0.9981 - val_loss: 0.2428 - val_acc: 0.9700
Epoch 292/300
2100/2100 [==============================] - 1s - loss: 0.0159 - acc: 0.9971 - val_loss: 0.3245 - val_acc: 0.9689
Epoch 293/300
2100/2100 [==============================] - 1s - loss: 0.0018 - acc: 0.9995 - val_loss: 0.2449 - val_acc: 0.9689
Epoch 294/300
2100/2100 [==============================] - 1s - loss: 0.0083 - acc: 0.9976 - val_loss: 0.2247 - val_acc: 0.9722
Epoch 295/300
2100/2100 [==============================] - 1s - loss: 0.0117 - acc: 0.9971 - val_loss: 0.2324 - val_acc: 0.9644
Epoch 296/300
2100/2100 [==============================] - 1s - loss: 0.0064 - acc: 0.9990 - val_loss: 0.2346 - val_acc: 0.9689
Epoch 297/300
2100/2100 [==============================] - 1s - loss: 0.0086 - acc: 0.9986 - val_loss: 0.2559 - val_acc: 0.9678
Epoch 298/300
2100/2100 [==============================] - 1s - loss: 0.0135 - acc: 0.9976 - val_loss: 0.2727 - val_ac

In [7]:
#Now let's process Telegu datas.
Telegu_datX = np.load('Telegu.npz')['arr_0']
Telegu_datY = np.load('Telegu.npz')['arr_1']

Telegu_datX = (Telegu_datX/255).astype('float32')
Telegu_datX = Telegu_datX.reshape((Telegu_datX.shape[0],1,32,32))
Telegu_datY = utils.np_utils.to_categorical(Telegu_datY,10)

Telegu_datX.shape,Telegu_datY.shape

((3000, 1, 32, 32), (3000, 10))

In [17]:
model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
#MLP weight flepath saving, we're usung thecallback loss to save the best weight model.  

weight_path="Telegu-transfer-main-weight.h5"

if(isfile(weight_path)):
    model.load_weights(weight_path)

checkpoint = ModelCheckpoint(weight_path, monitor='acc', verbose=1, save_best_only=True, mode='max')

#Some other parameter will be added later for finetune
callbacks_list = [checkpoint]

hist = model.fit(Telegu_datX,Telegu_datY, batch_size=100, epochs=300,
          verbose=1,validation_split=.3,callbacks=callbacks_list)

Train on 2100 samples, validate on 900 samples
Epoch 1/300
2100/2100 [==============================] - 1s - loss: 5.9078 - acc: 0.1010 - val_loss: 2.3710 - val_acc: 0.0956
Epoch 2/300
2100/2100 [==============================] - 1s - loss: 2.9017 - acc: 0.0910 - val_loss: 2.3233 - val_acc: 0.0956
Epoch 3/300
2100/2100 [==============================] - 1s - loss: 2.7984 - acc: 0.1124 - val_loss: 2.3128 - val_acc: 0.1011
Epoch 4/300
2100/2100 [==============================] - 1s - loss: 2.7531 - acc: 0.1048 - val_loss: 2.3125 - val_acc: 0.1000
Epoch 5/300
2100/2100 [==============================] - 1s - loss: 2.7045 - acc: 0.1038 - val_loss: 2.3046 - val_acc: 0.1000
Epoch 6/300
2100/2100 [==============================] - 1s - loss: 2.6910 - acc: 0.1033 - val_loss: 2.3070 - val_acc: 0.1000
Epoch 7/300
2100/2100 [==============================] - 1s - loss: 2.7030 - acc: 0.1029 - val_loss: 2.3080 - val_acc: 0.0989
Epoch 8/300
2100/2100 [==============================] - 1s - loss: 2.6

2100/2100 [==============================] - 1s - loss: 1.1476 - acc: 0.6043 - val_loss: 2.7692 - val_acc: 0.1011
Epoch 60/300
2100/2100 [==============================] - 1s - loss: 1.1413 - acc: 0.6129 - val_loss: 2.7020 - val_acc: 0.1022
Epoch 61/300
2100/2100 [==============================] - 1s - loss: 1.0322 - acc: 0.6481 - val_loss: 2.9250 - val_acc: 0.0944
Epoch 62/300
2100/2100 [==============================] - 1s - loss: 1.0499 - acc: 0.6462 - val_loss: 2.7429 - val_acc: 0.1044
Epoch 63/300
2100/2100 [==============================] - 1s - loss: 1.0305 - acc: 0.6452 - val_loss: 2.9738 - val_acc: 0.1000
Epoch 64/300
2100/2100 [==============================] - 1s - loss: 0.9571 - acc: 0.6781 - val_loss: 2.8950 - val_acc: 0.0956
Epoch 65/300
2100/2100 [==============================] - 1s - loss: 0.9614 - acc: 0.6790 - val_loss: 2.8151 - val_acc: 0.0933
Epoch 66/300
2100/2100 [==============================] - 1s - loss: 0.9279 - acc: 0.6843 - val_loss: 2.9237 - val_acc: 0.10

2100/2100 [==============================] - 1s - loss: 0.2888 - acc: 0.9029 - val_loss: 3.8803 - val_acc: 0.0922
Epoch 120/300
2100/2100 [==============================] - 1s - loss: 0.3232 - acc: 0.8976 - val_loss: 4.1334 - val_acc: 0.1011
Epoch 121/300
2100/2100 [==============================] - 1s - loss: 0.2824 - acc: 0.9143 - val_loss: 4.0616 - val_acc: 0.0933
Epoch 122/300
2100/2100 [==============================] - 1s - loss: 0.3005 - acc: 0.8967 - val_loss: 3.9783 - val_acc: 0.1111
Epoch 123/300
2100/2100 [==============================] - 1s - loss: 0.3255 - acc: 0.9057 - val_loss: 3.6956 - val_acc: 0.0967
Epoch 124/300
2100/2100 [==============================] - 1s - loss: 0.2609 - acc: 0.9110 - val_loss: 4.2829 - val_acc: 0.0956
Epoch 125/300
2100/2100 [==============================] - 1s - loss: 0.2796 - acc: 0.9067 - val_loss: 4.1119 - val_acc: 0.0833
Epoch 126/300
2100/2100 [==============================] - 1s - loss: 0.2428 - acc: 0.9219 - val_loss: 3.9556 - val_ac

2100/2100 [==============================] - 1s - loss: 0.1325 - acc: 0.9548 - val_loss: 4.6759 - val_acc: 0.0922
Epoch 184/300
2100/2100 [==============================] - 1s - loss: 0.1350 - acc: 0.9567 - val_loss: 4.2364 - val_acc: 0.0911
Epoch 185/300
2100/2100 [==============================] - 1s - loss: 0.1284 - acc: 0.9567 - val_loss: 4.7726 - val_acc: 0.0856
Epoch 186/300
2100/2100 [==============================] - 1s - loss: 0.1372 - acc: 0.9510 - val_loss: 4.3429 - val_acc: 0.0967
Epoch 187/300
2100/2100 [==============================] - 1s - loss: 0.1394 - acc: 0.9552 - val_loss: 4.7063 - val_acc: 0.0989
Epoch 188/300
2100/2100 [==============================] - 1s - loss: 0.1328 - acc: 0.9562 - val_loss: 4.4684 - val_acc: 0.0956
Epoch 189/300
2100/2100 [==============================] - 1s - loss: 0.1415 - acc: 0.9581 - val_loss: 4.4895 - val_acc: 0.0978
Epoch 190/300
2100/2100 [==============================] - 1s - loss: 0.1126 - acc: 0.9705 - val_loss: 4.4188 - val_ac

2100/2100 [==============================] - 1s - loss: 0.0683 - acc: 0.9752 - val_loss: 5.6453 - val_acc: 0.0989
Epoch 250/300
2100/2100 [==============================] - 1s - loss: 0.0861 - acc: 0.9700 - val_loss: 4.9954 - val_acc: 0.1011
Epoch 251/300
2100/2100 [==============================] - 1s - loss: 0.0999 - acc: 0.9690 - val_loss: 5.1893 - val_acc: 0.0933
Epoch 252/300
2100/2100 [==============================] - 1s - loss: 0.1000 - acc: 0.9695 - val_loss: 4.1212 - val_acc: 0.0933
Epoch 253/300
2100/2100 [==============================] - 1s - loss: 0.0986 - acc: 0.9690 - val_loss: 4.5816 - val_acc: 0.0922
Epoch 254/300
2100/2100 [==============================] - 1s - loss: 0.0678 - acc: 0.9786 - val_loss: 5.3425 - val_acc: 0.0900
Epoch 255/300
2100/2100 [==============================] - 1s - loss: 0.0844 - acc: 0.9743 - val_loss: 4.4879 - val_acc: 0.0933
Epoch 256/300
2100/2100 [==============================] - 1s - loss: 0.0881 - acc: 0.9671 - val_loss: 4.7701 - val_ac

# Now we will check transfer traning and model layer activations.

###  from bangla to arabic and vice versa

In [8]:
for layers in feature_layers:
    layers.trainable = False

model = Sequential(feature_layers+classification_layers)
model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 30, 30)        576       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 30, 30)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 28, 28)        36864     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 28, 28)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 28, 28)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 26, 26)        36864     
_________________________________________________________________
activation_3 (Activation)    (None, 64, 26, 26)        0         
__________

In [9]:
model.load_weights('Arabic-transfer-main-weight.h5')
model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 30, 30)        576       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 30, 30)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 28, 28)        36864     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 28, 28)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 28, 28)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 26, 26)        36864     
_________________________________________________________________
activation_3 (Activation)    (None, 64, 26, 26)        0         
__________

In [10]:
from keras.callbacks import ModelCheckpoint,TensorBoard
from os.path import isfile
#MLP weight flepath saving, we're usung the callback loss to save the best weight model.  
weight_path="Arabic-to-bangla.h5"

checkpoint = ModelCheckpoint(weight_path, monitor='acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./logs-A2B', histogram_freq=0,
                          write_graph=True, write_images=True)
#Some other parameter will be added later for finetune
callbacks_list = [checkpoint,tensorboard]

model.fit(Bangla_datX, Bangla_datY, batch_size=32, epochs=100,
          verbose=1,validation_split=.3,callbacks=callbacks_list)

Train on 4200 samples, validate on 1800 samples
Epoch 1/100
4200/4200 [==============================] - 6s - loss: 3.6071 - acc: 0.4550 - val_loss: 0.6027 - val_acc: 0.8144
Epoch 2/100
4200/4200 [==============================] - 2s - loss: 0.9218 - acc: 0.7369 - val_loss: 0.3988 - val_acc: 0.8811
Epoch 3/100
4200/4200 [==============================] - 2s - loss: 0.6155 - acc: 0.8171 - val_loss: 0.3387 - val_acc: 0.9061
Epoch 4/100
4200/4200 [==============================] - 2s - loss: 0.4756 - acc: 0.8598 - val_loss: 0.3367 - val_acc: 0.9006
Epoch 5/100
4200/4200 [==============================] - 2s - loss: 0.4015 - acc: 0.8793 - val_loss: 0.3112 - val_acc: 0.9161
Epoch 6/100
4200/4200 [==============================] - 2s - loss: 0.3374 - acc: 0.8933 - val_loss: 0.2778 - val_acc: 0.9272
Epoch 7/100
4200/4200 [==============================] - 2s - loss: 0.2812 - acc: 0.9129 - val_loss: 0.2784 - val_acc: 0.9289
Epoch 8/100
4200/4200 [==============================] - 2s - loss: 0.

4200/4200 [==============================] - 2s - loss: 0.0411 - acc: 0.9879 - val_loss: 0.3564 - val_acc: 0.9433
Epoch 30/100
4200/4200 [==============================] - 2s - loss: 0.0436 - acc: 0.9860 - val_loss: 0.3611 - val_acc: 0.9439
Epoch 31/100
4200/4200 [==============================] - 2s - loss: 0.0308 - acc: 0.9883 - val_loss: 0.3448 - val_acc: 0.9472
Epoch 32/100
4200/4200 [==============================] - 2s - loss: 0.0384 - acc: 0.9883 - val_loss: 0.3547 - val_acc: 0.9428
Epoch 33/100
4200/4200 [==============================] - 2s - loss: 0.0444 - acc: 0.9879 - val_loss: 0.3632 - val_acc: 0.9472
Epoch 34/100
4200/4200 [==============================] - 2s - loss: 0.0388 - acc: 0.9902 - val_loss: 0.3660 - val_acc: 0.9428
Epoch 35/100
4200/4200 [==============================] - 2s - loss: 0.0344 - acc: 0.9905 - val_loss: 0.3870 - val_acc: 0.9406
Epoch 36/100
4200/4200 [==============================] - 2s - loss: 0.0398 - acc: 0.9893 - val_loss: 0.3680 - val_acc: 0.94

4200/4200 [==============================] - 2s - loss: 0.0196 - acc: 0.9931 - val_loss: 0.4114 - val_acc: 0.9467
Epoch 62/100
4200/4200 [==============================] - 2s - loss: 0.0236 - acc: 0.9933 - val_loss: 0.4016 - val_acc: 0.9489
Epoch 63/100
4200/4200 [==============================] - 2s - loss: 0.0244 - acc: 0.9945 - val_loss: 0.4075 - val_acc: 0.9461
Epoch 64/100
4200/4200 [==============================] - 2s - loss: 0.0179 - acc: 0.9945 - val_loss: 0.4023 - val_acc: 0.9461
Epoch 65/100
4200/4200 [==============================] - 2s - loss: 0.0200 - acc: 0.9943 - val_loss: 0.3942 - val_acc: 0.9500
Epoch 66/100
4200/4200 [==============================] - 2s - loss: 0.0180 - acc: 0.9940 - val_loss: 0.3927 - val_acc: 0.9472
Epoch 67/100
4200/4200 [==============================] - 2s - loss: 0.0111 - acc: 0.9967 - val_loss: 0.3958 - val_acc: 0.9467
Epoch 68/100
4200/4200 [==============================] - 2s - loss: 0.0154 - acc: 0.9948 - val_loss: 0.4050 - val_acc: 0.94

4200/4200 [==============================] - 2s - loss: 0.0111 - acc: 0.9964 - val_loss: 0.4244 - val_acc: 0.9456
Epoch 95/100
4200/4200 [==============================] - 2s - loss: 0.0144 - acc: 0.9960 - val_loss: 0.4401 - val_acc: 0.9450
Epoch 96/100
4200/4200 [==============================] - 2s - loss: 0.0115 - acc: 0.9967 - val_loss: 0.4666 - val_acc: 0.9444
Epoch 97/100
4200/4200 [==============================] - 2s - loss: 0.0106 - acc: 0.9957 - val_loss: 0.4696 - val_acc: 0.9450
Epoch 98/100
4200/4200 [==============================] - 2s - loss: 0.0170 - acc: 0.9957 - val_loss: 0.4351 - val_acc: 0.9456
Epoch 99/100
4200/4200 [==============================] - 2s - loss: 0.0074 - acc: 0.9983 - val_loss: 0.4340 - val_acc: 0.9494
Epoch 100/100
4200/4200 [==============================] - 2s - loss: 0.0075 - acc: 0.9967 - val_loss: 0.4134 - val_acc: 0.9506


In [11]:
for layers in feature_layers:
    layers.trainable = False

model = Sequential(feature_layers+classification_layers)
model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
model.load_weights('Bangla-transfer-main-weight.h5')
model.summary()

from keras.callbacks import ModelCheckpoint,TensorBoard
from os.path import isfile
#MLP weight flepath saving, we're usung the callback loss to save the best weight model.  
weight_path="Bangla-to-Arabic.h5"

checkpoint = ModelCheckpoint(weight_path, monitor='acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./logs-B2A', histogram_freq=0,
                          write_graph=True, write_images=True)
#Some other parameter will be added later for finetune
callbacks_list = [checkpoint,tensorboard]

model.fit(Arabic_datX, Arabic_datY, batch_size=32, epochs=100,
          verbose=1,validation_split=.3,callbacks=callbacks_list)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 30, 30)        576       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 30, 30)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 28, 28)        36864     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 28, 28)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 28, 28)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 26, 26)        36864     
_________________________________________________________________
activation_3 (Activation)    (None, 64, 26, 26)        0         
__________

2100/2100 [==============================] - 1s - loss: 0.0592 - acc: 0.9862 - val_loss: 0.4501 - val_acc: 0.9278
Epoch 18/100
2100/2100 [==============================] - 1s - loss: 0.0615 - acc: 0.9886 - val_loss: 0.4398 - val_acc: 0.9356
Epoch 19/100
2100/2100 [==============================] - 1s - loss: 0.0408 - acc: 0.9876 - val_loss: 0.4720 - val_acc: 0.9322
Epoch 20/100
2100/2100 [==============================] - 1s - loss: 0.0442 - acc: 0.9867 - val_loss: 0.4260 - val_acc: 0.9433
Epoch 21/100
2100/2100 [==============================] - 1s - loss: 0.0466 - acc: 0.9895 - val_loss: 0.4427 - val_acc: 0.9367
Epoch 22/100
2100/2100 [==============================] - 1s - loss: 0.0500 - acc: 0.9890 - val_loss: 0.4242 - val_acc: 0.9389
Epoch 23/100
2100/2100 [==============================] - 1s - loss: 0.0614 - acc: 0.9852 - val_loss: 0.4231 - val_acc: 0.9433
Epoch 24/100
2100/2100 [==============================] - 1s - loss: 0.0535 - acc: 0.9862 - val_loss: 0.4249 - val_acc: 0.93

2100/2100 [==============================] - 1s - loss: 0.0264 - acc: 0.9952 - val_loss: 0.4459 - val_acc: 0.9422
Epoch 50/100
2100/2100 [==============================] - 1s - loss: 0.0124 - acc: 0.9952 - val_loss: 0.4565 - val_acc: 0.9411
Epoch 51/100
2100/2100 [==============================] - 1s - loss: 0.0368 - acc: 0.9938 - val_loss: 0.4680 - val_acc: 0.9389
Epoch 52/100
2100/2100 [==============================] - 1s - loss: 0.0230 - acc: 0.9952 - val_loss: 0.4481 - val_acc: 0.9400
Epoch 53/100
2100/2100 [==============================] - 1s - loss: 0.0109 - acc: 0.9957 - val_loss: 0.4627 - val_acc: 0.9444
Epoch 54/100
2100/2100 [==============================] - 1s - loss: 0.0147 - acc: 0.9948 - val_loss: 0.4952 - val_acc: 0.9456
Epoch 55/100
2100/2100 [==============================] - 1s - loss: 0.0179 - acc: 0.9957 - val_loss: 0.4496 - val_acc: 0.9456
Epoch 56/100
2100/2100 [==============================] - 1s - loss: 0.0264 - acc: 0.9957 - val_loss: 0.4782 - val_acc: 0.94

2100/2100 [==============================] - 1s - loss: 0.0038 - acc: 0.9990 - val_loss: 0.4286 - val_acc: 0.9456
Epoch 84/100
2100/2100 [==============================] - 1s - loss: 0.0078 - acc: 0.9971 - val_loss: 0.4532 - val_acc: 0.9444
Epoch 85/100
2100/2100 [==============================] - 1s - loss: 0.0056 - acc: 0.9981 - val_loss: 0.4693 - val_acc: 0.9456
Epoch 86/100
2100/2100 [==============================] - 1s - loss: 0.0109 - acc: 0.9952 - val_loss: 0.4585 - val_acc: 0.9411
Epoch 87/100
2100/2100 [==============================] - 1s - loss: 0.0199 - acc: 0.9962 - val_loss: 0.4836 - val_acc: 0.9422
Epoch 88/100
2100/2100 [==============================] - 1s - loss: 0.0212 - acc: 0.9948 - val_loss: 0.4946 - val_acc: 0.9389
Epoch 89/100
2100/2100 [==============================] - 1s - loss: 0.0144 - acc: 0.9981 - val_loss: 0.4973 - val_acc: 0.9389
Epoch 90/100
2100/2100 [==============================] - 1s - loss: 0.0148 - acc: 0.9976 - val_loss: 0.4990 - val_acc: 0.93

In [12]:
for layers in feature_layers:
    layers.trainable = False

model = Sequential(feature_layers+classification_layers)
model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
model.load_weights('Bangla-transfer-main-weight.h5')
model.summary()

from keras.callbacks import ModelCheckpoint,TensorBoard
from os.path import isfile
#MLP weight flepath saving, we're usung the callback loss to save the best weight model.  
weight_path="Bangla-to-Hindi.h5"

checkpoint = ModelCheckpoint(weight_path, monitor='acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./logs-B2H', histogram_freq=0,
                          write_graph=True, write_images=True)
#Some other parameter will be added later for finetune
callbacks_list = [checkpoint,tensorboard]

model.fit(Hindi_datX, Hindi_datY, batch_size=32, epochs=100,
          verbose=1,validation_split=.3,callbacks=callbacks_list)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 30, 30)        576       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 30, 30)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 28, 28)        36864     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 28, 28)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 28, 28)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 26, 26)        36864     
_________________________________________________________________
activation_3 (Activation)    (None, 64, 26, 26)        0         
__________

2100/2100 [==============================] - 1s - loss: 0.2282 - acc: 0.9348 - val_loss: 0.1971 - val_acc: 0.9567
Epoch 17/100
2100/2100 [==============================] - 1s - loss: 0.1731 - acc: 0.9524 - val_loss: 0.2033 - val_acc: 0.9478
Epoch 18/100
2100/2100 [==============================] - 1s - loss: 0.1636 - acc: 0.9514 - val_loss: 0.1985 - val_acc: 0.9533
Epoch 19/100
2100/2100 [==============================] - 1s - loss: 0.1953 - acc: 0.9481 - val_loss: 0.1867 - val_acc: 0.9511
Epoch 20/100
2100/2100 [==============================] - 1s - loss: 0.1475 - acc: 0.9571 - val_loss: 0.2049 - val_acc: 0.9489
Epoch 21/100
2100/2100 [==============================] - 1s - loss: 0.1612 - acc: 0.9562 - val_loss: 0.2067 - val_acc: 0.9567
Epoch 22/100
2100/2100 [==============================] - 1s - loss: 0.1288 - acc: 0.9600 - val_loss: 0.1871 - val_acc: 0.9556
Epoch 23/100
2100/2100 [==============================] - 1s - loss: 0.1451 - acc: 0.9571 - val_loss: 0.1787 - val_acc: 0.96

2100/2100 [==============================] - 1s - loss: 0.0397 - acc: 0.9881 - val_loss: 0.2285 - val_acc: 0.9667
Epoch 49/100
2100/2100 [==============================] - 1s - loss: 0.0532 - acc: 0.9886 - val_loss: 0.2294 - val_acc: 0.9644
Epoch 50/100
2100/2100 [==============================] - 1s - loss: 0.0281 - acc: 0.9867 - val_loss: 0.2289 - val_acc: 0.9667
Epoch 51/100
2100/2100 [==============================] - 1s - loss: 0.0324 - acc: 0.9895 - val_loss: 0.2273 - val_acc: 0.9644
Epoch 52/100
2100/2100 [==============================] - 1s - loss: 0.0415 - acc: 0.9900 - val_loss: 0.2329 - val_acc: 0.9633
Epoch 53/100
2100/2100 [==============================] - 1s - loss: 0.0209 - acc: 0.9943 - val_loss: 0.2326 - val_acc: 0.9633
Epoch 54/100
2100/2100 [==============================] - 1s - loss: 0.0221 - acc: 0.9943 - val_loss: 0.2542 - val_acc: 0.9622
Epoch 55/100
2100/2100 [==============================] - 1s - loss: 0.0293 - acc: 0.9895 - val_loss: 0.2866 - val_acc: 0.96

2100/2100 [==============================] - 1s - loss: 0.0246 - acc: 0.9957 - val_loss: 0.2510 - val_acc: 0.9678
Epoch 81/100
2100/2100 [==============================] - 1s - loss: 0.0120 - acc: 0.9962 - val_loss: 0.2545 - val_acc: 0.9656
Epoch 82/100
2100/2100 [==============================] - 1s - loss: 0.0123 - acc: 0.9952 - val_loss: 0.2584 - val_acc: 0.9656
Epoch 83/100
2100/2100 [==============================] - 1s - loss: 0.0305 - acc: 0.9924 - val_loss: 0.2369 - val_acc: 0.9656
Epoch 84/100
2100/2100 [==============================] - 1s - loss: 0.0271 - acc: 0.9919 - val_loss: 0.2276 - val_acc: 0.9667
Epoch 85/100
2100/2100 [==============================] - 1s - loss: 0.0211 - acc: 0.9976 - val_loss: 0.2435 - val_acc: 0.9667
Epoch 86/100
2100/2100 [==============================] - 1s - loss: 0.0207 - acc: 0.9943 - val_loss: 0.2529 - val_acc: 0.9656
Epoch 87/100
2100/2100 [==============================] - 1s - loss: 0.0176 - acc: 0.9938 - val_loss: 0.2453 - val_acc: 0.96

In [14]:
for layers in feature_layers:
    layers.trainable = False

model = Sequential(feature_layers+classification_layers)
model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
model.load_weights('Hindi-transfer-main-weight.h5')
model.summary()

from keras.callbacks import ModelCheckpoint,TensorBoard
from os.path import isfile
#MLP weight flepath saving, we're usung the callback loss to save the best weight model.  
weight_path="Hindi-to-Bangla.h5"

checkpoint = ModelCheckpoint(weight_path, monitor='acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./logs-H2B', histogram_freq=0,
                          write_graph=True, write_images=True)
#Some other parameter will be added later for finetune
callbacks_list = [checkpoint,tensorboard]

model.fit(Bangla_datX, Bangla_datY, batch_size=32, epochs=100,
          verbose=1,validation_split=.3,callbacks=callbacks_list)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 30, 30)        576       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 30, 30)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 28, 28)        36864     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 28, 28)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 28, 28)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 26, 26)        36864     
_________________________________________________________________
activation_3 (Activation)    (None, 64, 26, 26)        0         
__________

4200/4200 [==============================] - 2s - loss: 0.0310 - acc: 0.9936 - val_loss: 0.3544 - val_acc: 0.9633
Epoch 18/100
4200/4200 [==============================] - 2s - loss: 0.0275 - acc: 0.9938 - val_loss: 0.3480 - val_acc: 0.9611
Epoch 19/100
4200/4200 [==============================] - 2s - loss: 0.0301 - acc: 0.9943 - val_loss: 0.3728 - val_acc: 0.9606
Epoch 20/100
4200/4200 [==============================] - 2s - loss: 0.0298 - acc: 0.9938 - val_loss: 0.3777 - val_acc: 0.9639
Epoch 21/100
4200/4200 [==============================] - 2s - loss: 0.0205 - acc: 0.9957 - val_loss: 0.3792 - val_acc: 0.9633
Epoch 22/100
4200/4200 [==============================] - 2s - loss: 0.0162 - acc: 0.9967 - val_loss: 0.3714 - val_acc: 0.9622
Epoch 23/100
4200/4200 [==============================] - 2s - loss: 0.0186 - acc: 0.9952 - val_loss: 0.3813 - val_acc: 0.9583
Epoch 24/100
4200/4200 [==============================] - 2s - loss: 0.0166 - acc: 0.9943 - val_loss: 0.3873 - val_acc: 0.96

4200/4200 [==============================] - 2s - loss: 0.0090 - acc: 0.9983 - val_loss: 0.4009 - val_acc: 0.9644
Epoch 51/100
4200/4200 [==============================] - 2s - loss: 0.0072 - acc: 0.9979 - val_loss: 0.4168 - val_acc: 0.9633
Epoch 52/100
4200/4200 [==============================] - 2s - loss: 0.0078 - acc: 0.9990 - val_loss: 0.4098 - val_acc: 0.9617
Epoch 53/100
4200/4200 [==============================] - 2s - loss: 0.0067 - acc: 0.9988 - val_loss: 0.4454 - val_acc: 0.9583
Epoch 54/100
4200/4200 [==============================] - 2s - loss: 0.0143 - acc: 0.9969 - val_loss: 0.4258 - val_acc: 0.9594
Epoch 55/100
4200/4200 [==============================] - 2s - loss: 0.0159 - acc: 0.9967 - val_loss: 0.4066 - val_acc: 0.9639
Epoch 56/100
4200/4200 [==============================] - 2s - loss: 0.0097 - acc: 0.9974 - val_loss: 0.4124 - val_acc: 0.9639
Epoch 57/100
4200/4200 [==============================] - 2s - loss: 0.0119 - acc: 0.9979 - val_loss: 0.4305 - val_acc: 0.96

4200/4200 [==============================] - 2s - loss: 9.6865e-04 - acc: 0.9998 - val_loss: 0.4311 - val_acc: 0.9639
Epoch 84/100
4200/4200 [==============================] - 2s - loss: 0.0109 - acc: 0.9983 - val_loss: 0.4409 - val_acc: 0.9611
Epoch 85/100
4200/4200 [==============================] - 2s - loss: 0.0059 - acc: 0.9986 - val_loss: 0.4237 - val_acc: 0.9639
Epoch 86/100
4200/4200 [==============================] - 2s - loss: 0.0047 - acc: 0.9988 - val_loss: 0.4226 - val_acc: 0.9650
Epoch 87/100
4200/4200 [==============================] - 2s - loss: 0.0082 - acc: 0.9979 - val_loss: 0.4164 - val_acc: 0.9661
Epoch 88/100
4200/4200 [==============================] - 2s - loss: 0.0046 - acc: 0.9986 - val_loss: 0.4236 - val_acc: 0.9633
Epoch 89/100
4200/4200 [==============================] - 2s - loss: 0.0015 - acc: 0.9995 - val_loss: 0.4280 - val_acc: 0.9656
Epoch 90/100
4200/4200 [==============================] - 2s - loss: 0.0036 - acc: 0.9995 - val_loss: 0.4259 - val_acc: 

In [15]:
for layers in feature_layers:
    layers.trainable = False

model = Sequential(feature_layers+classification_layers)
model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
model.load_weights('Hindi-transfer-main-weight.h5')
model.summary()

from keras.callbacks import ModelCheckpoint,TensorBoard
from os.path import isfile
#MLP weight flepath saving, we're usung the callback loss to save the best weight model.  
weight_path="Hindi-to-Arabic.h5"

checkpoint = ModelCheckpoint(weight_path, monitor='acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./logs-H2A', histogram_freq=0,
                          write_graph=True, write_images=True)
#Some other parameter will be added later for finetune
callbacks_list = [checkpoint,tensorboard]

model.fit(Arabic_datX, Arabic_datY, batch_size=32, epochs=100,
          verbose=1,validation_split=.3,callbacks=callbacks_list)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 30, 30)        576       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 30, 30)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 28, 28)        36864     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 28, 28)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 28, 28)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 26, 26)        36864     
_________________________________________________________________
activation_3 (Activation)    (None, 64, 26, 26)        0         
__________

2100/2100 [==============================] - 1s - loss: 0.1024 - acc: 0.9767 - val_loss: 0.3192 - val_acc: 0.9300
Epoch 17/100
2100/2100 [==============================] - 1s - loss: 0.0816 - acc: 0.9810 - val_loss: 0.3703 - val_acc: 0.9333
Epoch 18/100
2100/2100 [==============================] - 1s - loss: 0.1124 - acc: 0.9743 - val_loss: 0.3121 - val_acc: 0.9367
Epoch 19/100
2100/2100 [==============================] - 1s - loss: 0.0729 - acc: 0.9824 - val_loss: 0.3410 - val_acc: 0.9344
Epoch 20/100
2100/2100 [==============================] - 1s - loss: 0.0878 - acc: 0.9814 - val_loss: 0.3615 - val_acc: 0.9344
Epoch 21/100
2100/2100 [==============================] - 1s - loss: 0.0693 - acc: 0.9805 - val_loss: 0.3884 - val_acc: 0.9333
Epoch 22/100
2100/2100 [==============================] - 1s - loss: 0.0582 - acc: 0.9852 - val_loss: 0.3355 - val_acc: 0.9378
Epoch 23/100
2100/2100 [==============================] - 1s - loss: 0.0730 - acc: 0.9824 - val_loss: 0.3426 - val_acc: 0.93

2100/2100 [==============================] - 1s - loss: 0.0332 - acc: 0.9910 - val_loss: 0.4287 - val_acc: 0.9456
Epoch 49/100
2100/2100 [==============================] - 1s - loss: 0.0364 - acc: 0.9910 - val_loss: 0.4767 - val_acc: 0.9411
Epoch 50/100
2100/2100 [==============================] - 1s - loss: 0.0199 - acc: 0.9957 - val_loss: 0.4584 - val_acc: 0.9400
Epoch 51/100
2100/2100 [==============================] - 1s - loss: 0.0434 - acc: 0.9948 - val_loss: 0.4576 - val_acc: 0.9456
Epoch 52/100
2100/2100 [==============================] - 1s - loss: 0.0278 - acc: 0.9938 - val_loss: 0.4765 - val_acc: 0.9400
Epoch 53/100
2100/2100 [==============================] - 1s - loss: 0.0354 - acc: 0.9900 - val_loss: 0.4558 - val_acc: 0.9422
Epoch 54/100
2100/2100 [==============================] - 1s - loss: 0.0319 - acc: 0.9933 - val_loss: 0.4633 - val_acc: 0.9422
Epoch 55/100
2100/2100 [==============================] - 1s - loss: 0.0147 - acc: 0.9952 - val_loss: 0.4412 - val_acc: 0.94

2100/2100 [==============================] - 1s - loss: 0.0211 - acc: 0.9957 - val_loss: 0.4803 - val_acc: 0.9467
Epoch 82/100
2100/2100 [==============================] - 1s - loss: 0.0259 - acc: 0.9967 - val_loss: 0.4844 - val_acc: 0.9456
Epoch 83/100
2100/2100 [==============================] - 1s - loss: 0.0343 - acc: 0.9924 - val_loss: 0.4551 - val_acc: 0.9478
Epoch 84/100
2100/2100 [==============================] - 1s - loss: 0.0262 - acc: 0.9957 - val_loss: 0.4581 - val_acc: 0.9489
Epoch 85/100
2100/2100 [==============================] - 1s - loss: 0.0155 - acc: 0.9971 - val_loss: 0.4596 - val_acc: 0.9467
Epoch 86/100
2100/2100 [==============================] - 1s - loss: 0.0214 - acc: 0.9957 - val_loss: 0.4612 - val_acc: 0.9489
Epoch 87/100
2100/2100 [==============================] - 1s - loss: 0.0234 - acc: 0.9957 - val_loss: 0.4195 - val_acc: 0.9522
Epoch 88/100
2100/2100 [==============================] - 1s - loss: 0.0207 - acc: 0.9948 - val_loss: 0.4347 - val_acc: 0.95

In [16]:
for layers in feature_layers:
    layers.trainable = False

model = Sequential(feature_layers+classification_layers)
model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
model.load_weights('Arabic-transfer-main-weight.h5')
model.summary()

from keras.callbacks import ModelCheckpoint,TensorBoard
from os.path import isfile
#MLP weight flepath saving, we're usung the callback loss to save the best weight model.  
weight_path="Arabic-to-HIndi.h5"

checkpoint = ModelCheckpoint(weight_path, monitor='acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./logs-A2H', histogram_freq=0,
                          write_graph=True, write_images=True)
#Some other parameter will be added later for finetune
callbacks_list = [checkpoint,tensorboard]

model.fit(Hindi_datX, Hindi_datY, batch_size=32, epochs=100,
          verbose=1,validation_split=.3,callbacks=callbacks_list)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 30, 30)        576       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 30, 30)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 28, 28)        36864     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 28, 28)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 28, 28)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 26, 26)        36864     
_________________________________________________________________
activation_3 (Activation)    (None, 64, 26, 26)        0         
__________

2100/2100 [==============================] - 1s - loss: 0.2509 - acc: 0.9271 - val_loss: 0.2701 - val_acc: 0.9389
Epoch 17/100
2100/2100 [==============================] - 1s - loss: 0.2265 - acc: 0.9352 - val_loss: 0.2473 - val_acc: 0.9444
Epoch 18/100
2100/2100 [==============================] - 1s - loss: 0.1980 - acc: 0.9352 - val_loss: 0.2736 - val_acc: 0.9433
Epoch 19/100
2100/2100 [==============================] - 1s - loss: 0.1770 - acc: 0.9462 - val_loss: 0.2948 - val_acc: 0.9367
Epoch 20/100
2100/2100 [==============================] - 1s - loss: 0.1383 - acc: 0.9557 - val_loss: 0.2662 - val_acc: 0.9433
Epoch 21/100
2100/2100 [==============================] - 1s - loss: 0.1237 - acc: 0.9562 - val_loss: 0.2744 - val_acc: 0.9444
Epoch 22/100
2100/2100 [==============================] - 1s - loss: 0.1521 - acc: 0.9490 - val_loss: 0.2645 - val_acc: 0.9478
Epoch 23/100
2100/2100 [==============================] - 1s - loss: 0.1230 - acc: 0.9614 - val_loss: 0.2674 - val_acc: 0.94

2100/2100 [==============================] - 1s - loss: 0.0497 - acc: 0.9833 - val_loss: 0.3221 - val_acc: 0.9467
Epoch 48/100
2100/2100 [==============================] - 1s - loss: 0.0414 - acc: 0.9862 - val_loss: 0.3492 - val_acc: 0.9444
Epoch 49/100
2100/2100 [==============================] - 1s - loss: 0.0385 - acc: 0.9890 - val_loss: 0.3442 - val_acc: 0.9467
Epoch 50/100
2100/2100 [==============================] - 1s - loss: 0.0449 - acc: 0.9857 - val_loss: 0.3444 - val_acc: 0.9478
Epoch 51/100
2100/2100 [==============================] - 1s - loss: 0.0631 - acc: 0.9810 - val_loss: 0.3206 - val_acc: 0.9489
Epoch 52/100
2100/2100 [==============================] - 1s - loss: 0.0469 - acc: 0.9886 - val_loss: 0.3604 - val_acc: 0.9456
Epoch 53/100
2100/2100 [==============================] - 1s - loss: 0.0294 - acc: 0.9890 - val_loss: 0.3335 - val_acc: 0.9456
Epoch 54/100
2100/2100 [==============================] - 1s - loss: 0.0273 - acc: 0.9919 - val_loss: 0.3710 - val_acc: 0.94

2100/2100 [==============================] - 1s - loss: 0.0229 - acc: 0.9929 - val_loss: 0.3660 - val_acc: 0.9544
Epoch 80/100
2100/2100 [==============================] - 1s - loss: 0.0238 - acc: 0.9938 - val_loss: 0.3651 - val_acc: 0.9544
Epoch 81/100
2100/2100 [==============================] - 1s - loss: 0.0207 - acc: 0.9919 - val_loss: 0.3508 - val_acc: 0.9500
Epoch 82/100
2100/2100 [==============================] - 1s - loss: 0.0235 - acc: 0.9919 - val_loss: 0.3469 - val_acc: 0.9500
Epoch 83/100
2100/2100 [==============================] - 1s - loss: 0.0229 - acc: 0.9952 - val_loss: 0.3547 - val_acc: 0.9478
Epoch 84/100
2100/2100 [==============================] - 1s - loss: 0.0135 - acc: 0.9948 - val_loss: 0.3508 - val_acc: 0.9522
Epoch 85/100
2100/2100 [==============================] - 1s - loss: 0.0240 - acc: 0.9938 - val_loss: 0.3640 - val_acc: 0.9533
Epoch 86/100
2100/2100 [==============================] - 1s - loss: 0.0156 - acc: 0.9938 - val_loss: 0.3570 - val_acc: 0.95